<a href="https://colab.research.google.com/github/Morsalah/M.Sc-Research-HRI-using-DIGIT-tactile-sensor/blob/main/gripper_control_keboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
import rospy
import actionlib
from robotiq_2f_gripper_msgs.msg import CommandRobotiqGripperAction, CommandRobotiqGripperGoal
from pynput import keyboard

# Default gripper parameters
position = 0.085  # Fully open
speed = 0.05
force = 100.0

def send_gripper_command(position, speed, force):
    """ Sends a command to the gripper with the specified position, speed, and force """
    client = actionlib.SimpleActionClient('/command_robotiq_action', CommandRobotiqGripperAction)

    rospy.loginfo("Waiting for the gripper action server...")
    client.wait_for_server()

    goal = CommandRobotiqGripperGoal()
    goal.position = position
    goal.speed = speed
    goal.force = force

    rospy.loginfo(f"Sending command - Position: {position:.2f}, Speed: {speed:.2f}, Force: {force:.2f}")
    client.send_goal(goal)
    client.wait_for_result()

    result = client.get_result()
    rospy.loginfo("Gripper response: {}".format(result))

def on_press(key):
    global position, speed, force

    try:
        if key.char == 'o':  # Open gripper
            position = 0.085
            rospy.loginfo("Opening gripper...")

        elif key.char == 'c':  # Close gripper
            position = 0.00
            rospy.loginfo("Closing gripper...")

        elif key.char == '+':  # Increase force
            force = min(100.0, force + 5)
            rospy.loginfo(f"Increasing force: {force}")

        elif key.char == '-':  # Decrease force
            force = max(10.0, force - 5)
            rospy.loginfo(f"Decreasing force: {force}")

        elif key.char == 's':  # Increase speed
            speed = min(0.1, speed + 0.01)
            rospy.loginfo(f"Increasing speed: {speed}")

        elif key.char == 'd':  # Decrease speed
            speed = max(0.01, speed - 0.01)
            rospy.loginfo(f"Decreasing speed: {speed}")

        send_gripper_command(position, speed, force)

    except AttributeError:
        pass

def main():
    rospy.init_node('gripper_keyboard_control')
    rospy.loginfo("Keyboard Control for Robotiq 2F Gripper")
    rospy.loginfo("Press 'o' to open, 'c' to close, '+' to increase force, '-' to decrease force")
    rospy.loginfo("Press 's' to increase speed, 'd' to decrease speed")
    rospy.loginfo("Press ESC to exit.")

    listener = keyboard.Listener(on_press=on_press)
    listener.start()
    rospy.spin()

if __name__ == '__main__':
    main()
